<a href="https://colab.research.google.com/github/shinichiromizuno/QueryMultiTopic/blob/master/Multi_Span_Selector.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Mount your Google Drive.
# Connect to GPU in Google Colab.

In [ ]:
################ Preprocess ################

In [ ]:
# Configure working directories.　We use the preprocessed dataset output of Multi-BERTSum by configuring it as the source directory.
work_dir =  '/content/drive/MyDrive/work_Multi_Span_Selector'
source_dir = '/content/drive/MyDrive/work_Multi_BERTSum'

In [ ]:
# Skipping Step1~5, as we use the preprocessed dataset output of Multi-BERTSum.

In [ ]:
# Install required packages and configure module path.
!pip install transformers
!pip install pyrouge
%env MODULE_PATH=/content/drive/MyDrive/QueryMultiTopic

     |████████████████████████████████| 4.0 MB 5.0 MB/s 
     |████████████████████████████████| 6.6 MB 17.5 MB/s 
     |████████████████████████████████| 895 kB 51.8 MB/s 
     |████████████████████████████████| 596 kB 45.9 MB/s 
     |████████████████████████████████| 77 kB 6.8 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
     |████████████████████████████████| 60 kB 3.2 MB/s 
  Created wheel for pyrouge: filename=pyrouge-0.1.3-py3-none-any.whl size=191621 sha256=07fc53d66afbd474031e0ce9f4eb8e015523314a7fe11db537a58d8e53b6025c
  Stored in directory: /root/.cache/pip/wheels/68/35/6a/ffb9a1f51b2b00fee42e7f67f5a5d8e10c67d048cda09ccd57
Successfully built pyrouge
env: MODULE_PATH=/content/drive/MyDrive/QueryMultiTopic


In [ ]:
# Step6. Adding Start Target Positions and End Target Positions
from glob import glob
import os
import shutil
import torch

dir = work_dir + '/pt_data'
if os.path.exists(dir):
  shutil.rmtree(dir)
os.mkdir(dir)

for k in range(17):
  sub_dir = dir + '/' + str(k+1)
  if not os.path.exists(sub_dir):
    os.mkdir(sub_dir)
  for data in sorted(glob(source_dir + '/pt_data/' + str(k+1) + '/*.pt')):
    dataset = torch.load(glob(data)[0])
    for j, d in enumerate(dataset['ds']):
      tgt_seq = dataset['tgts'][d['qry']][d['stpos']:d['enpos']]
      pos = [i for i, x in enumerate(tgt_seq) if x == 1]
      if pos:
        start_tgt_pos = min(pos)
        end_tgt_pos = max(pos)
      else:
        start_tgt_pos = len(tgt_seq) - 1
        end_tgt_pos = 0
      dataset['ds'][j]['tgt'] = [start_tgt_pos, end_tgt_pos]
    filename = data.split('/')[-1]
    torch.save(dataset, sub_dir + '/' + filename)

In [ ]:
# Check Number of Dataset by Query
from glob import glob
import torch
for i in range(1,17+1):
  ds_counts = []
  zero_sent_cnts = []
  z_cnt = 0
  o_cnt = 0
  for data_type in ['train', 'test', 'valid']:
    ds_list = glob(work_dir + '/pt_data/' + str(i) + '/' + data_type + '.*')
    ds_count = 0
    for ds in ds_list:
      dataset = torch.load(ds)
      ds_count += len(dataset['ds'])
      for d in dataset['ds']:
        z_cnt += len([j for j in d['tgt'] if j == 0])
        o_cnt += len([j for j in d['tgt'] if j == 1])
    ds_counts.append(ds_count)
  print(f'qry: {i}, train: {ds_counts[0]}, test: {ds_counts[1]}, valid: {ds_counts[2]}, ds_total: {ds_counts[0] + ds_counts[1] + ds_counts[2]}, zero_sent_in_ds: {z_cnt}, one_sent_in_ds: {o_cnt}')

qry: 1, train: 24286, test: 4912, valid: 5065, ds_total: 34263, zero_sent_in_ds: 34088, one_sent_in_ds: 36
qry: 2, train: 24286, test: 4912, valid: 5065, ds_total: 34263, zero_sent_in_ds: 34067, one_sent_in_ds: 34
qry: 3, train: 24286, test: 4912, valid: 5065, ds_total: 34263, zero_sent_in_ds: 33411, one_sent_in_ds: 122
qry: 4, train: 24286, test: 4912, valid: 5065, ds_total: 34263, zero_sent_in_ds: 33831, one_sent_in_ds: 75
qry: 5, train: 24286, test: 4912, valid: 5065, ds_total: 34263, zero_sent_in_ds: 33733, one_sent_in_ds: 71
qry: 6, train: 24286, test: 4912, valid: 5065, ds_total: 34263, zero_sent_in_ds: 33942, one_sent_in_ds: 54
qry: 7, train: 24286, test: 4912, valid: 5065, ds_total: 34263, zero_sent_in_ds: 33473, one_sent_in_ds: 124
qry: 8, train: 24286, test: 4912, valid: 5065, ds_total: 34263, zero_sent_in_ds: 33386, one_sent_in_ds: 148
qry: 9, train: 24286, test: 4912, valid: 5065, ds_total: 34263, zero_sent_in_ds: 33395, one_sent_in_ds: 129
qry: 10, train: 24286, test: 4912

In [ ]:
# Check Output
from glob import glob
import torch
dataset = torch.load(glob(work_dir + '/pt_data/12/*.1.Ahresty.2020.pt')[0])
print(dataset['src'])
print(dataset['tgts'])
print(len(dataset['ds']))
for d in dataset['ds']:
  print(d)
  break

['We aim to create an affluent society by pursuing each of these areas and integrating them .', 'Our corporate name , Ahresty , comes from “ RST . ”', 'It was created by linking together the pronunciation of the first letters in Research , Service , and Technology .', 'Research means continuous research , investigation , and development of new technologies , new markets , and new sales techniques .', 'Service means providing warm , attentive service through personal interaction .', 'Technology means truly excellent technology that incorporates both physical and soft aspects and is highly beneficial for society .', 'These three areas of Research , Service , and Technology can not be considered independently .', 'Both technology and a spirit of service are necessary to accomplish the research involved in R&D .', 'To explain it in another way , research , service , and technology are intricately linked and each supports the others .', 'It is an organic relationship in which each component

In [ ]:
# Preparing Data for Validation & Test
import json
import random
from glob import glob
import torch

src_doc_valid = []
tgt_list_valid = []
src_doc_test = []
tgt_list_test = []

for data in sorted(glob(source_dir + '/json_data_labeled/*')):
  filetype = data.split('/')[-1].split('.')[1]
  json_open = open(data)
  json_load = json.load(json_open)
  src_txt = json_load['src_txt']
  if filetype == 'valid':
    src_doc_valid.append(src_txt)
    tgt_list = []
    for i in range(1,18):
      tgt_list.append(json_load['tgt'+str(i)])
    tgt_list_valid.append(tgt_list)
  if filetype == 'test':
    src_doc_test.append(src_txt)
    tgt_list = []
    for i in range(1,18):
      tgt_list.append(json_load['tgt'+str(i)])
    tgt_list_test.append(tgt_list)

# Incorporate SDGs goal text in query text
sdgsdir = '/content/drive/MyDrive/DatasetESG/0-SDGs/'

all_query_txt = []
for goal in sorted(glob(sdgsdir + '/*')):
  query_txt = []
  with open(goal, mode='r') as f:
    for line in f:
      query_txt.append(line.strip())
      break
  all_query_txt.append(' '.join(query_txt))

dataset = {'src_doc_valid': src_doc_valid, 'tgt_list_valid': tgt_list_valid, 'src_doc_test': src_doc_test, 'tgt_list_test': tgt_list_test, 'all_query_txt': all_query_txt}
torch.save(dataset, work_dir + '/dataset.pt')  

In [ ]:
################ Training ################

In [ ]:
# Install Pytorch
!pip3 install torchtext==0.3.1
!pip3 install torch==1.6.0
!pip install tensorboardX

     |████████████████████████████████| 62 kB 874 kB/s 
  Attempting uninstall: torchtext
    Found existing installation: torchtext 0.11.0
    Uninstalling torchtext-0.11.0:
      Successfully uninstalled torchtext-0.11.0
     |████████████████████████████████| 748.8 MB 16 kB/s 
  Attempting uninstall: torch
    Found existing installation: torch 1.10.0+cu111
    Uninstalling torch-1.10.0+cu111:
      Successfully uninstalled torch-1.10.0+cu111
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchvision 0.11.1+cu111 requires torch==1.10.0, but you have torch 1.6.0 which is incompatible.
torchaudio 0.10.0+cu111 requires torch==1.10.0, but you have torch 1.6.0 which is incompatible.


     |████████████████████████████████| 125 kB 5.0 MB/s 


In [ ]:
# Training Models by Query
!rm -r $work_dir/models
!mkdir $work_dir/models
!mkdir logs

for idx in range(1, 18):
  %env BERT_DATA_PATH=$work_dir/pt_data/$idx/ 
  %env MODEL_PATH=$work_dir/models/proposed$idx
  %env LOG_PATH=/content/logs/proposed$idx
  !python $MODULE_PATH/src/train.py -mode train -summarizer proposedQA -dropout 0.1 -bert_data_path $BERT_DATA_PATH -model_path $MODEL_PATH -lr 2e-3 -visible_gpus 0  -gpu_ranks 0 -world_size 1 -report_every 10 -save_checkpoint_steps 1000 -batch_size 4000 -decay_method noam -train_steps 10000 -accum_count 2 -log_file $LOG_PATH -use_interval true -warmup_steps 10000 -ff_size 2048 -inter_layers 2 -heads 8

ストリーミング出力は最後の 5000 行に切り捨てられました。
[2022-04-04 00:08:02,378 INFO] Step 8190/10000; xent: 0.02; lr: 0.0000164;  35 docs/s;   6521 sec
[2022-04-04 00:08:08,632 INFO] Loading train dataset from /content/drive/MyDrive/work_proposed_qa/pt_data/7/train.70.Omron.2017.pt, number of examples: 96
[2022-04-04 00:08:10,243 INFO] Step 8200/10000; xent: 0.01; lr: 0.0000164;  35 docs/s;   6529 sec
[2022-04-04 00:08:14,237 INFO] Loading train dataset from /content/drive/MyDrive/work_proposed_qa/pt_data/7/train.105.Exeo.2020.pt, number of examples: 133
[2022-04-04 00:08:18,250 INFO] Step 8210/10000; xent: 0.02; lr: 0.0000164;  35 docs/s;   6537 sec
[2022-04-04 00:08:22,260 INFO] Loading train dataset from /content/drive/MyDrive/work_proposed_qa/pt_data/7/train.183.SRI.2019.pt, number of examples: 108
[2022-04-04 00:08:26,021 INFO] Step 8220/10000; xent: 0.02; lr: 0.0000164;  35 docs/s;   6545 sec
[2022-04-04 00:08:28,502 INFO] Loading train dataset from /content/drive/MyDrive/work_proposed_qa/pt_data/7/tr

In [ ]:
################ Validation ################

In [ ]:
# Validation
!cp $MODULE_PATH/bert_config_uncased_base_proposed.json ../bert_config_uncased_base.json
!rm -r $work_dir/results
!mkdir $work_dir/results
!rm -r $work_dir/logs
!mkdir $work_dir/logs
%env LOG_PATH=$work_dir/logs/proposed

for idx in range(1, 18):
  !mkdir $work_dir/results/$idx
  %env RESULT_PATH=$work_dir/results/$idx/ 
  %env BERT_DATA_PATH=$work_dir/pt_data/$idx/
  %env MODEL_PATH=$work_dir/models/proposed$idx
  !python $MODULE_PATH/src/train.py -mode valid -summarizer proposedQA -bert_data_path $BERT_DATA_PATH -visible_gpus 0  -gpu_ranks 0 -batch_size 512 -log_file $LOG_PATH  -result_path $RESULT_PATH -test_from $MODEL_PATH/model_step_21000.pt -block_trigram true
  !python $MODULE_PATH/src/train.py -mode valid -summarizer proposedQA -bert_data_path $BERT_DATA_PATH -visible_gpus 0  -gpu_ranks 0 -batch_size 512 -log_file $LOG_PATH  -result_path $RESULT_PATH -test_from $MODEL_PATH/model_step_22000.pt -block_trigram true
  !python $MODULE_PATH/src/train.py -mode valid -summarizer proposedQA -bert_data_path $BERT_DATA_PATH -visible_gpus 0  -gpu_ranks 0 -batch_size 512 -log_file $LOG_PATH  -result_path $RESULT_PATH -test_from $MODEL_PATH/model_step_23000.pt -block_trigram true
  !python $MODULE_PATH/src/train.py -mode valid -summarizer proposedQA -bert_data_path $BERT_DATA_PATH -visible_gpus 0  -gpu_ranks 0 -batch_size 512 -log_file $LOG_PATH  -result_path $RESULT_PATH -test_from $MODEL_PATH/model_step_24000.pt -block_trigram true

env: LOG_PATH=/content/drive/MyDrive/work_proposed_qa/logs/proposed
mkdir: cannot create directory ‘/content/drive/MyDrive/work_proposed_qa/results/1’: File exists
env: RESULT_PATH=/content/drive/MyDrive/work_proposed_qa/results/1/
env: BERT_DATA_PATH=/content/drive/MyDrive/work_proposed_qa/pt_data/1/
env: MODEL_PATH=/content/drive/MyDrive/work_proposed_qa/models/proposed1
[2022-04-22 21:48:34,512 INFO] Loading checkpoint from /content/drive/MyDrive/work_proposed_qa/models/proposed1/model_step_21000.pt
Namespace(accum_count=1, batch_size=512, bert_config_path='../bert_config_uncased_base.json', bert_data_path='/content/drive/MyDrive/work_proposed_qa/pt_data/1/', beta1=0.9, beta2=0.999, block_trigram=True, dataset='', decay_method='', dropout=0.1, ff_size=2048, gpu_ranks=[0], heads=8, hidden_size=128, inter_layers=2, log_file='/content/drive/MyDrive/work_proposed_qa/logs/proposed', lr=1, max_grad_norm=0, mode='valid', model_path='../models/', optim='adam', param_init=0, param_init_gloro

In [ ]:
# Exploring Optimal Hyper-Parameters
def gen_optim_list(pt_optim_dicts_all, qry, priority):
  optim_dicts_all = []
  for pt_optim_dicts in pt_optim_dicts_all:
    optim_dicts = {}
    optim_dicts[qry] = {}
    for pt_optim_dict in pt_optim_dicts[qry]:
      stpos = pt_optim_dict['stpos']
      enpos = pt_optim_dict['enpos']
      start_logits, end_logits = pt_optim_dict['pred'].split(1, dim=-1)
      st_pred = torch.argmax(start_logits.squeeze())
      en_pred = torch.argmax(end_logits.squeeze())
      pred = [1 if i >= st_pred and i <= en_pred else 0 for i in range(enpos - stpos)]
      for j in range(stpos, enpos):
        st_idx = j - stpos
        en_idx = enpos - 1 - j
        if priority == 'center':          
          pred_eval = st_idx * en_idx
        elif priority == 'top':
          pred_eval = (en_idx) ** 2
        elif priority == 'bottom':
          pred_eval = (st_idx) ** 2
        else:
          None
        if optim_dicts[qry].get(j):
          if optim_dicts[qry][j][-1] < pred_eval:
            optim_dicts[qry][j] = [pred[st_idx], pred_eval]
        else:
          optim_dicts[qry][j] = [pred[st_idx], pred_eval]
    optim_dicts_all.append(optim_dicts)
  return optim_dicts_all

In [ ]:
# Showing F1 Score and Accuracy in all and by qry
import numpy as np
from sklearn.metrics import f1_score, accuracy_score

def show_statistics_valid(all_predited_list, all_tgt_list):
  by_qry_tgt_list = []
  by_qry_pred_list = []
  best_results = []
  for i in range(17):
    by_qry_tgt = []
    for data in all_tgt_list:
      by_qry_tgt += data[i]
    by_qry_pred = []
    for data in all_predited_list:
      by_qry_pred += data[i]
    f1 = f1_score(by_qry_pred, by_qry_tgt)
    acc = accuracy_score(by_qry_pred, by_qry_tgt)
    print(f'Goal{i+1}, f1_score: {round(f1, 3)}, accuracy: {round(acc, 3)}')
    best_results.append((round(f1, 3), round(acc, 3)))

    by_qry_tgt_list.append(by_qry_tgt)
    by_qry_pred_list.append(by_qry_pred)
  
  tgt_list_all = []
  pred_list_all = []
  for by_qry_tgt in by_qry_tgt_list:
    tgt_list_all += by_qry_tgt
  for by_qry_pred in by_qry_pred_list:
    pred_list_all += by_qry_pred
  
  f1_all = f1_score(pred_list_all, tgt_list_all)
  acc_all = accuracy_score(pred_list_all, tgt_list_all)
  print(f'Total, f1_score: {round(f1_all, 3)}, accuracy: {round(acc_all, 3)}')
  return best_results

In [ ]:
import torch
from glob import glob
import numpy as np

def validate_execute(validation_list):
  best_results_list = []
  for step, align in validation_list:
    print(f'step:{step}, align:{align}')
    dataset = torch.load(glob(work_dir + '/dataset.pt')[0])
    optim_list_all = []
    for i in range(1, 18):
      pt_optim_dicts_all = torch.load(glob(work_dir + '/results/' + str(i) + '/valid.step' + str(step) + '.optim_dicts_all.pt')[0])
      optim_dicts_all = gen_optim_list(pt_optim_dicts_all, str(i), align)

      optim_list_by_qry = []
      for optim_dict in optim_dicts_all:
        optim_list_by_qry.append(torch.tensor([optim_dict[str(i)][j][0] for j in sorted(optim_dict[str(i)])]))
      optim_list_all.append(optim_list_by_qry)
    
    optim_list_all_t = np.array(optim_list_all).T.tolist()
    best_results = show_statistics_valid(optim_list_all_t, dataset['tgt_list_valid'])
    best_results_list.append((step, align, best_results))
  return best_results_list

In [ ]:
validation_list = [(i, j) for j in ['center', 'top', 'bottom'] for i in range(10000, 24001, 1000)]
best_results_list = validate_execute(validation_list)
torch.save(best_results_list, work_dir + '/best_results_list.pt')  

step:10000, align:center


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:20: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'Tensor', or assign to an empty array created with `np.empty(correct_shape, dtype=object)`.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:20: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.


Goal1, f1_score: 0.062, accuracy: 0.988
Goal2, f1_score: 0.329, accuracy: 0.991
Goal3, f1_score: 0.164, accuracy: 0.941
Goal4, f1_score: 0.315, accuracy: 0.978
Goal5, f1_score: 0.437, accuracy: 0.959
Goal6, f1_score: 0.222, accuracy: 0.99
Goal7, f1_score: 0.411, accuracy: 0.953
Goal8, f1_score: 0.451, accuracy: 0.921
Goal9, f1_score: 0.037, accuracy: 0.941
Goal10, f1_score: 0.241, accuracy: 0.975
Goal11, f1_score: 0.365, accuracy: 0.968
Goal12, f1_score: 0.426, accuracy: 0.898
Goal13, f1_score: 0.376, accuracy: 0.874
Goal14, f1_score: 0.331, accuracy: 0.963
Goal15, f1_score: 0.402, accuracy: 0.96
Goal16, f1_score: 0.179, accuracy: 0.976
Goal17, f1_score: 0.059, accuracy: 0.943
Total, f1_score: 0.335, accuracy: 0.954
step:11000, align:center


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:20: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'Tensor', or assign to an empty array created with `np.empty(correct_shape, dtype=object)`.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:20: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.


Goal1, f1_score: 0.054, accuracy: 0.986
Goal2, f1_score: 0.194, accuracy: 0.991
Goal3, f1_score: 0.302, accuracy: 0.925
Goal4, f1_score: 0.436, accuracy: 0.972
Goal5, f1_score: 0.392, accuracy: 0.965
Goal6, f1_score: 0.309, accuracy: 0.976
Goal7, f1_score: 0.256, accuracy: 0.954
Goal8, f1_score: 0.428, accuracy: 0.929
Goal9, f1_score: 0.032, accuracy: 0.943
Goal10, f1_score: 0.153, accuracy: 0.978
Goal11, f1_score: 0.136, accuracy: 0.965
Goal12, f1_score: 0.381, accuracy: 0.947
Goal13, f1_score: 0.406, accuracy: 0.947
Goal14, f1_score: 0.119, accuracy: 0.977
Goal15, f1_score: 0.084, accuracy: 0.969
Goal16, f1_score: 0.012, accuracy: 0.975
Goal17, f1_score: 0.017, accuracy: 0.946
Total, f1_score: 0.274, accuracy: 0.961
step:12000, align:center


...

Goal1, f1_score: 0.108, accuracy: 0.988
Goal2, f1_score: 0.261, accuracy: 0.991
Goal3, f1_score: 0.313, accuracy: 0.939
Goal4, f1_score: 0.443, accuracy: 0.979
Goal5, f1_score: 0.357, accuracy: 0.966
Goal6, f1_score: 0.364, accuracy: 0.987
Goal7, f1_score: 0.409, accuracy: 0.956
Goal8, f1_score: 0.376, accuracy: 0.93
Goal9, f1_score: 0.036, accuracy: 0.94
Goal10, f1_score: 0.373, accuracy: 0.978
Goal11, f1_score: 0.347, accuracy: 0.968
Goal12, f1_score: 0.445, accuracy: 0.946
Goal13, f1_score: 0.417, accuracy: 0.956
Goal14, f1_score: 0.17, accuracy: 0.976
Goal15, f1_score: 0.309, accuracy: 0.973
Goal16, f1_score: 0.351, accuracy: 0.975
Goal17, f1_score: 0.19, accuracy: 0.944
Total, f1_score: 0.33, accuracy: 0.964
step:23000, align:bottom


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:20: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'Tensor', or assign to an empty array created with `np.empty(correct_shape, dtype=object)`.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:20: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.


Goal1, f1_score: 0.168, accuracy: 0.989
Goal2, f1_score: 0.191, accuracy: 0.99
Goal3, f1_score: 0.308, accuracy: 0.941
Goal4, f1_score: 0.413, accuracy: 0.982
Goal5, f1_score: 0.347, accuracy: 0.964
Goal6, f1_score: 0.406, accuracy: 0.988
Goal7, f1_score: 0.424, accuracy: 0.959
Goal8, f1_score: 0.403, accuracy: 0.931
Goal9, f1_score: 0.249, accuracy: 0.942
Goal10, f1_score: 0.349, accuracy: 0.977
Goal11, f1_score: 0.367, accuracy: 0.966
Goal12, f1_score: 0.449, accuracy: 0.949
Goal13, f1_score: 0.39, accuracy: 0.952
Goal14, f1_score: 0.437, accuracy: 0.966
Goal15, f1_score: 0.373, accuracy: 0.974
Goal16, f1_score: 0.352, accuracy: 0.978
Goal17, f1_score: 0.155, accuracy: 0.947
Total, f1_score: 0.356, accuracy: 0.964
step:24000, align:bottom


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:20: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'Tensor', or assign to an empty array created with `np.empty(correct_shape, dtype=object)`.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:20: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.


Goal1, f1_score: 0.098, accuracy: 0.988
Goal2, f1_score: 0.243, accuracy: 0.989
Goal3, f1_score: 0.307, accuracy: 0.939
Goal4, f1_score: 0.443, accuracy: 0.982
Goal5, f1_score: 0.427, accuracy: 0.966
Goal6, f1_score: 0.405, accuracy: 0.989
Goal7, f1_score: 0.493, accuracy: 0.955
Goal8, f1_score: 0.339, accuracy: 0.932
Goal9, f1_score: 0.329, accuracy: 0.934
Goal10, f1_score: 0.262, accuracy: 0.978
Goal11, f1_score: 0.213, accuracy: 0.964
Goal12, f1_score: 0.481, accuracy: 0.948
Goal13, f1_score: 0.203, accuracy: 0.954
Goal14, f1_score: 0.255, accuracy: 0.976
Goal15, f1_score: 0.403, accuracy: 0.974
Goal16, f1_score: 0.371, accuracy: 0.976
Goal17, f1_score: 0.213, accuracy: 0.944
Total, f1_score: 0.345, accuracy: 0.964


In [ ]:
# Showing Optimal Hyperparameters (Model, Threshold, and Alignment)
import numpy as np
best_results_list = torch.load(glob(work_dir + '/best_results_list.pt')[0])
params_list = []
for i in range(17):
  best_result = best_results_list[np.argmax([b[i][0] for steps, align, b in best_results_list])]
  print('Goal' + str(i+1), best_result[0], best_result[1])
  params_list.append((i+1, best_result[0], best_result[1]))

Goal1 16000 bottom
Goal2 19000 bottom
Goal3 16000 bottom
Goal4 13000 center
Goal5 20000 center
Goal6 18000 center
Goal7 19000 center
Goal8 10000 center
Goal9 12000 center
Goal10 16000 center
Goal11 20000 top
Goal12 24000 top
Goal13 16000 bottom
Goal14 23000 center
Goal15 16000 bottom
Goal16 17000 top
Goal17 20000 bottom


In [ ]:
# Showing F1 Score and Accuracy in all an by qry
import numpy as np
from sklearn.metrics import f1_score, accuracy_score

def show_statistics_valid_test(all_predited_list, all_tgt_list):
  by_qry_tgt_list = []
  by_qry_pred_list = []
  for i in range(17):
    by_qry_tgt = []
    for data in all_tgt_list:
      by_qry_tgt += data[i]
    by_qry_pred = []
    for data in all_predited_list:
      by_qry_pred += data[i]
    f1 = f1_score(by_qry_pred, by_qry_tgt)
    acc = accuracy_score(by_qry_pred, by_qry_tgt)
    print(f'Goal{i+1}, f1_score: {round(f1, 3)}, accuracy: {round(acc, 3)}')

    by_qry_tgt_list.append(by_qry_tgt)
    by_qry_pred_list.append(by_qry_pred)
  
  tgt_list_all = []
  pred_list_all = []
  for by_qry_tgt in by_qry_tgt_list:
    tgt_list_all += by_qry_tgt
  for by_qry_pred in by_qry_pred_list:
    pred_list_all += by_qry_pred
  
  f1_all = f1_score(pred_list_all, tgt_list_all)
  acc_all = accuracy_score(pred_list_all, tgt_list_all)
  print(f'Total, f1_score: {round(f1_all, 3)}, accuracy: {round(acc_all, 3)}')

In [ ]:
import torch
from glob import glob
import numpy as np

def valid_test_execute(params_list, vt_flag):
  dataset = torch.load(glob(work_dir + '/dataset.pt')[0])
  optim_list_all = []
  for i, step, align in params_list:
    pt_optim_dicts_all = torch.load(glob(work_dir + '/results/' + str(i) + '/' + vt_flag + '.step' + str(step) + '.optim_dicts_all.pt')[0])
    optim_dicts_all = gen_optim_list(pt_optim_dicts_all, str(i), align)
    optim_list_by_qry = []
    for optim_dict in optim_dicts_all:
      optim_list_by_qry.append(torch.tensor([optim_dict[str(i)][j][0] for j in sorted(optim_dict[str(i)])]))
    optim_list_all.append(optim_list_by_qry)

  optim_list_all_t = np.array(optim_list_all).T.tolist()
  data_type = 'tgt_list_' + vt_flag
  show_statistics_valid_test(optim_list_all_t, dataset[data_type])

In [ ]:
valid_test_execute(params_list, 'valid')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:16: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'Tensor', or assign to an empty array created with `np.empty(correct_shape, dtype=object)`.
  app.launch_new_instance()
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:16: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  app.launch_new_instance()


Goal1, f1_score: 0.176, accuracy: 0.988
Goal2, f1_score: 0.392, accuracy: 0.991
Goal3, f1_score: 0.38, accuracy: 0.9
Goal4, f1_score: 0.518, accuracy: 0.983
Goal5, f1_score: 0.482, accuracy: 0.959
Goal6, f1_score: 0.425, accuracy: 0.99
Goal7, f1_score: 0.515, accuracy: 0.943
Goal8, f1_score: 0.451, accuracy: 0.921
Goal9, f1_score: 0.332, accuracy: 0.918
Goal10, f1_score: 0.382, accuracy: 0.974
Goal11, f1_score: 0.402, accuracy: 0.967
Goal12, f1_score: 0.483, accuracy: 0.948
Goal13, f1_score: 0.502, accuracy: 0.94
Goal14, f1_score: 0.459, accuracy: 0.969
Goal15, f1_score: 0.529, accuracy: 0.97
Goal16, f1_score: 0.446, accuracy: 0.979
Goal17, f1_score: 0.226, accuracy: 0.943
Total, f1_score: 0.43, accuracy: 0.958


In [ ]:
################ Test ################

In [ ]:
!cp $MODULE_PATH/bert_config_uncased_base_proposed.json ../bert_config_uncased_base.json

%env LOG_PATH=$work_dir/logs/proposed

for idx, steps, _ in params_list:
  %env RESULT_PATH=$work_dir/results/$idx/ 
  %env BERT_DATA_PATH=$work_dir/pt_data/$idx/
  %env MODEL_PATH=$work_dir/models/proposed$idx
  model_name = 'model_step_' + str(steps) + '.pt'
  %env MODEL_NAME=$model_name
  !python $MODULE_PATH/src/train.py -mode test -summarizer proposedQA -bert_data_path $BERT_DATA_PATH -visible_gpus 0  -gpu_ranks 0 -batch_size 512 -log_file $LOG_PATH  -result_path $RESULT_PATH -test_from $MODEL_PATH/$MODEL_NAME -block_trigram true

env: LOG_PATH=/content/drive/MyDrive/work_proposed_qa/logs/proposed
env: RESULT_PATH=/content/drive/MyDrive/work_proposed_qa/results/1/
env: BERT_DATA_PATH=/content/drive/MyDrive/work_proposed_qa/pt_data/1/
env: MODEL_PATH=/content/drive/MyDrive/work_proposed_qa/models/proposed1
env: MODEL_NAME=model_step_16000.pt
[2022-04-23 02:11:39,798 INFO] Loading checkpoint from /content/drive/MyDrive/work_proposed_qa/models/proposed1/model_step_16000.pt
Namespace(accum_count=1, batch_size=512, bert_config_path='../bert_config_uncased_base.json', bert_data_path='/content/drive/MyDrive/work_proposed_qa/pt_data/1/', beta1=0.9, beta2=0.999, block_trigram=True, dataset='', decay_method='', dropout=0.1, ff_size=2048, gpu_ranks=[0], heads=8, hidden_size=128, inter_layers=2, log_file='/content/drive/MyDrive/work_proposed_qa/logs/proposed', lr=1, max_grad_norm=0, mode='test', model_path='../models/', optim='adam', param_init=0, param_init_glorot=True, recall_eval=False, report_every=1, report_rouge=True,

In [ ]:
# Showing Test Results
valid_test_execute(params_list, 'test')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:16: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'Tensor', or assign to an empty array created with `np.empty(correct_shape, dtype=object)`.
  app.launch_new_instance()
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:16: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  app.launch_new_instance()


Goal1, f1_score: 0.142, accuracy: 0.989
Goal2, f1_score: 0.075, accuracy: 0.99
Goal3, f1_score: 0.271, accuracy: 0.892
Goal4, f1_score: 0.434, accuracy: 0.979
Goal5, f1_score: 0.37, accuracy: 0.95
Goal6, f1_score: 0.519, accuracy: 0.986
Goal7, f1_score: 0.433, accuracy: 0.939
Goal8, f1_score: 0.349, accuracy: 0.909
Goal9, f1_score: 0.309, accuracy: 0.918
Goal10, f1_score: 0.235, accuracy: 0.961
Goal11, f1_score: 0.244, accuracy: 0.958
Goal12, f1_score: 0.41, accuracy: 0.943
Goal13, f1_score: 0.47, accuracy: 0.921
Goal14, f1_score: 0.379, accuracy: 0.96
Goal15, f1_score: 0.395, accuracy: 0.965
Goal16, f1_score: 0.316, accuracy: 0.975
Goal17, f1_score: 0.164, accuracy: 0.949
Total, f1_score: 0.35, accuracy: 0.952
